In [8]:
include("packages.jl")
include("quickscore_preparation.jl")
include("quickscore_algorithm.jl")

# LOAD SOME RANDOM VARIABLES previn,pfminneg,pfmin FOR A CERTAIN m=17 and run the quickscore
file = jldopen("variables/m_18.jld", "r")
previn = file["previn"]
pfminneg = file["pfminneg"]
pfmin = file["pfmin"]
close(file)

# m,n = size(pfmin)
# prev = repeat(previn',inner=(n+1,1)); for i in 1:n prev[i+1,i]=1 end 
# prevend = vcat(1,previn) # prevend are the prevalences 1, P(d_1=1),... P(d_n=1) and is needed to multiply at the end
# prevminneg = prev.*pfminneg'

pfplus_bigfloat,posterior_bigfloat,dt = quickscore(previn, pfmin, pfminneg,false); 
# pfplus_prod_bigfloat,posterior_prod_bigfloat,dt = quickscore(previn, pfmin, pfminneg,false); 
# pfplus_prod,posterior_prod,dt = quickscore(previn, pfmin, pfminneg,false); 
# pfplus,posterior,dt = quickscore(previn, pfmin, pfminneg,false); 

m=18
flip
0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.00010085105895996094 
1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.00017189979553222656 
2: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0024139881134033203 
3: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0027740001678466797 
4: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0033299922943115234 
5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.004363059997558594 
6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.006532907485961914 
7: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.010828971862792969 
8: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.027920007705688477 
9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.044113874435424805 
10: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0] --> 0.09161996841430664 


11: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] --> 0.16587591171264648 
12: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] --> 0.30801892280578613 


13: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] --> 0.5980269908905029 
14: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] --> 1.181765079498291 


15: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] --> 2.3315608501434326 
16: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] --> 4.671578884124756 


17: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] --> 9.415632009506226 
18: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 19.033061981201172 



Total running time was 19.033061981201172 



([2.7217561537895563e-10, 2.0511392497674086e-10, 1.2656272180993668e-10, 1.609940050197045e-10, 9.990845536945667e-11, 1.1392632071559848e-10, 4.662273502581047e-10, 2.3331189530264446e-10, 8.432527251337266e-11, 8.541489217306353e-11  …  7.75453816827601e-11, 9.437953749687894e-11, 3.2986720885190557e-10, 1.8044558122015664e-10, 6.792086967724945e-10, 1.3605535432005891e-10, 1.908471706948793e-10, 1.3942699807743299e-10, 2.790684820275051e-10, 5.935760863406449e-10], [0.02911513068890243, 0.01097062247980413, 0.01743922847719698, 0.012885447236388941, 0.016244015741358957, 0.06235495903364059, 0.030045517479015276, 0.010869634075051732, 0.00882821152000157, 0.044116890772693415  …  0.007207602188520982, 0.007732171037148874, 0.029583424860576715, 0.016643200395751994, 0.09510964578228936, 0.016426171161670933, 0.02121727515460768, 0.011196132081267135, 0.024798391441625562, 0.05108128323429383], Any[0.00010085105895996094, 0.00017189979553222656, 0.0024139881134033203, 0.002774000167

In [3]:
matlab_dir = "/Users/sam/Documents/MATLAB/Wim/"
matfile = matopen(joinpath(matlab_dir,"variables_julia/test.mat"),"r") 
posterior_matlab = read(matfile, "posterior")[1,:]
close(matfile) 


In [61]:
# posterior_bigfloat - posterior_matlab
# posterior - posterior_matlab
# posterior - posterior_bigfloat
# pfplus - pfplus_bigfloat
# pfplus_prod_bigfloat - pfplus_prod
# maximum(abs,pfplus - pfplus_prod)
# maximum(abs,posterior-posterior_matlab)
# sum(abs,posterior-posterior_bigfloat)



101-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [60]:
previn

100-element Vector{Float64}:
 0.03863427903780465
 0.02359253879681072
 0.02948266764302406
 0.035103180386766256
 0.038807757091241464
 0.03640176694377803
 0.03505032265338364
 0.03508378040346216
 0.028131205718594447
 0.03279209704483333
 ⋮
 0.022298354771244
 0.024409479482574062
 0.02510381955024049
 0.038112772248468374
 0.032860178613262424
 0.0302588972152099
 0.021855983282309324
 0.02418588226802133
 0.023422573817540676